In [ ]:
# Cell 1 — Configuration and Imports (FREE INSTRUCTION MODELS)
import os
import json
import random
import re
import collections
import numpy as np
import torch
import matplotlib.pyplot as plt
from datasets import load_dataset, Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,  # Changed from Seq2Seq to CausalLM
    Trainer,  # Changed from Seq2SeqTrainer
    TrainingArguments,  # Changed from Seq2SeqTrainingArguments
    DataCollatorForLanguageModeling  # Changed collator
)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    PeftModel
)
from sklearn.model_selection import train_test_split
import transformers

# 🚀 BEST FREE INSTRUCTION MODELS FOR HUMANIZATION
FREE_MODELS = [
    {
        "name": "microsoft/DialoGPT-large",
        "description": "Excellent for conversational humanization",
        "size": "774M",
        "strengths": "Natural dialogue, fast training"
    },
    {
        "name": "mistralai/Mistral-7B-v0.1", 
        "description": "High-quality instruction following",
        "size": "7B",
        "strengths": "Excellent reasoning, no gating"
    },
    {
        "name": "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
        "description": "Fast and efficient for limited resources",
        "size": "1.1B", 
        "strengths": "Very fast, good quality"
    },
    {
        "name": "microsoft/GRIN-MoE",
        "description": "Mixture of experts for diverse styles",
        "size": "2.7B",
        "strengths": "Multiple writing styles"
    }
]

# AUTO-SELECT BEST MODEL BASED ON AVAILABLE MEMORY
def select_best_model():
    """Automatically select the best free model based on system capabilities"""
    
    if torch.cuda.is_available():
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
        print(f"🖥️  GPU Memory: {gpu_memory:.1f} GB")
        
        if gpu_memory >= 20:
            # High memory - use best quality
            return "mistralai/Mistral-7B-v0.1"
        elif gpu_memory >= 8:
            # Medium memory - balanced option
            return "microsoft/DialoGPT-large"
        else:
            # Limited memory - efficient option
            return "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
    else:
        print("💻 Using CPU - selecting efficient model")
        return "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

MODEL_NAME = select_best_model()
print(f"\n🤖 AUTO-SELECTED MODEL: {MODEL_NAME}")

# Find model info
model_info = next((m for m in FREE_MODELS if m["name"] == MODEL_NAME), None)
if model_info:
    print(f"📊 Size: {model_info['size']}")
    print(f"💪 Strengths: {model_info['strengths']}")
    print(f"📝 Description: {model_info['description']}")

# Alternative models if auto-selection fails
FALLBACK_MODELS = [
    "microsoft/DialoGPT-medium",  # 355M - Always works
    "gpt2-large",                 # 774M - Reliable fallback
    "distilgpt2"                  # 82M - Minimal option
]

# OPTIMAL DATASETS FOR INSTRUCTION-FOLLOWING HUMANIZATION  
DATASET_OPTIONS = [
    "HuggingFaceH4/ultrachat_200k",  # High-quality conversational data
    "microsoft/orca-math-word-problems-200k",  # Natural problem descriptions
    "argilla/distilabel-math-preference-dpo"  # Human preference data
]

# OPTIMIZED SETTINGS FOR FREE MODELS
MAX_LENGTH = 1024  # Reduced for efficiency
BATCH_SIZE = 2  # Conservative for memory
GRADIENT_ACCUMULATION = 4  # Effective batch size = 8
EPOCHS = 3  # More epochs for smaller models
LR = 5e-5  # Higher LR for faster convergence
SEED = 42
MAX_SAMPLES = 8000  # Focused training set
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# OPTIMIZED LORA CONFIG FOR FREE MODELS
LORA_R = 16  # Balanced rank
LORA_ALPHA = 32  # 2x rank
LORA_DROPOUT = 0.05  # Low dropout

# Set seeds
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print("\n🎉" * 30)
print("FREE INSTRUCTION MODEL HUMANIZATION SETUP")
print("🎉" * 30)

print(f"\n✅ SELECTED FREE MODEL: {MODEL_NAME}")
print("🚀 No authentication required!")
print("⚡ Optimized for humanization tasks")
print("💾 Memory-efficient configuration")

print(f"\n🔧 DEVICE: {DEVICE}")
print(f"🧠 PyTorch: {torch.__version__}")
print(f"⚡ CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🎯 CUDA version: {torch.version.cuda}")
    print(f"🖥️  GPU: {torch.cuda.get_device_name(0)}")
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"💾 GPU Memory: {gpu_memory:.1f} GB")

print(f"\n📈 OPTIMIZED CONFIG:")
print(f"  Max length: {MAX_LENGTH}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Gradient accumulation: {GRADIENT_ACCUMULATION}")
print(f"  Effective batch: {BATCH_SIZE * GRADIENT_ACCUMULATION}")
print(f"  Epochs: {EPOCHS}")
print(f"  Learning rate: {LR}")
print(f"  LoRA rank: {LORA_R}")
print(f"  Max samples: {MAX_SAMPLES:,}")

print(f"\n💡 WHY FREE MODELS WORK GREAT:")
print("  ✅ No authentication barriers")
print("  ✅ Optimized configurations provided")
print("  ✅ Fast training and inference")
print("  ✅ Excellent humanization results")
print("  ✅ Multiple fallback options")

print(f"\n📋 AVAILABLE FREE MODELS:")
for model in FREE_MODELS:
    status = "🟢 SELECTED" if model["name"] == MODEL_NAME else "⚪"
    print(f"  {status} {model['name']} ({model['size']})")
    print(f"      {model['description']}")

print(f"\n🚀 Ready to load {MODEL_NAME}!")

In [ ]:
# Cell 2 — Load Instruction-Following Dataset for Llama
print("🔄 Loading high-quality conversational dataset for Llama-3.1...")

# STRATEGY: Use instruction-following datasets that Llama understands natively
datasets_to_try = [
    {
        "name": "HuggingFaceH4/ultrachat_200k",
        "description": "High-quality multi-turn conversations - perfect for humanization",
        "field": "messages"
    },
    {
        "name": "microsoft/orca-math-word-problems-200k", 
        "description": "Natural problem descriptions with human explanations",
        "field": "question"
    },
    {
        "name": "argilla/distilabel-math-preference-dpo",
        "description": "Human preference data for natural responses",
        "field": "instruction"
    },
    {
        "name": "tatsu-lab/alpaca_cleaned",
        "description": "Clean instruction-following dataset",
        "field": "instruction"
    }
]

ds = None
dataset_info = None

for dataset_config in datasets_to_try:
    try:
        print(f"\n🔄 Trying {dataset_config['name']}...")
        print(f"   Purpose: {dataset_config['description']}")
        
        # Load dataset
        ds_all = load_dataset(dataset_config['name'])
        
        # Get training split
        if "train" in ds_all:
            ds = ds_all["train"]
        elif "train_sft" in ds_all:
            ds = ds_all["train_sft"]  # Some datasets use this
        else:
            # Use first available split
            split_name = list(ds_all.keys())[0]
            ds = ds_all[split_name]
            print(f"   Using split: {split_name}")
        
        dataset_info = dataset_config
        print(f"✅ Successfully loaded {dataset_config['name']}")
        print(f"📦 Dataset size: {len(ds):,} samples")
        break
        
    except Exception as e:
        print(f"❌ Could not load {dataset_config['name']}: {str(e)[:100]}...")
        continue

# Fallback to reliable paraphrase dataset
if ds is None:
    print("\n🔄 Falling back to ChatGPT paraphrases (converted for instruction format)...")
    try:
        ds_all = load_dataset("humarin/chatgpt-paraphrases")
        ds = ds_all["train"]
        dataset_info = {
            "name": "humarin/chatgpt-paraphrases",
            "description": "Paraphrase pairs converted to instruction format",
            "field": "text"
        }
        print(f"✅ Successfully loaded fallback dataset")
    except Exception as e:
        raise ValueError(f"❌ Could not load any dataset! Last error: {e}")

print(f"\n📊 DATASET LOADED: {dataset_info['name']}")
print(f"📝 Description: {dataset_info['description']}")
print(f"📦 Dataset size: {len(ds):,} samples")
print(f"🏷️  Available columns: {ds.column_names}")

# Show sample record
sample = ds[0]
print(f"\n🔍 SAMPLE RECORD STRUCTURE:")
for key in sample.keys():
    value = sample[key]
    if isinstance(value, (list, dict)):
        print(f"  {key}: {type(value).__name__} with {len(value) if hasattr(value, '__len__') else '?'} items")
        if isinstance(value, list) and len(value) > 0:
            print(f"    Sample item: {str(value[0])[:100]}...")
        elif isinstance(value, dict):
            print(f"    Keys: {list(value.keys())}")
    else:
        value_preview = str(value)[:150]
        if len(str(value)) > 150:
            value_preview += "..."
        print(f"  {key}: {value_preview}")

print(f"\n🎯 Primary field: '{dataset_info['field']}'")
print(f"✅ Dataset ready for Llama-3.1 instruction processing!")

In [ ]:
# Cell 2.5 — Inspect Dataset Structure (UPDATED FOR ACTUAL STRUCTURE)
print("=" * 60)
print("DATASET STRUCTURE INSPECTION")
print("=" * 60)

sample = ds[0]
print(f"\nColumns: {ds.column_names}")
print(f"\nSample record keys: {list(sample.keys())}")

# Inspect actual fields that exist
for key, value in sample.items():
    print(f"\n--- '{key}' field ---")
    print(f"Type: {type(value)}")
    
    if isinstance(value, str):
        preview = value[:200] if len(value) > 200 else value
        print(f"Value: {preview}...")
    elif isinstance(value, list):
        print(f"List length: {len(value)}")
        if len(value) > 0:
            print(f"First item type: {type(value[0])}")
            if isinstance(value[0], dict):
                print(f"First item keys: {list(value[0].keys())}")
                print(f"First item: {value[0]}")
            else:
                preview = str(value[0])[:150] if len(str(value[0])) > 150 else str(value[0])
                print(f"First item: {preview}")
            
            if len(value) > 1:
                if isinstance(value[1], dict):
                    print(f"Second item: {value[1]}")
                else:
                    preview = str(value[1])[:150] if len(str(value[1])) > 150 else str(value[1])
                    print(f"Second item: {preview}")
    elif isinstance(value, dict):
        print(f"Dict keys: {list(value.keys())}")
        for k, v in value.items():
            preview = str(v)[:100] if len(str(v)) > 100 else str(v)
            print(f"  {k}: {preview}")
    else:
        print(f"Value: {value}")

print(f"\n" + "=" * 60)
print("ANALYSIS FOR LLAMA PROCESSING:")
print("=" * 60)

# Determine the best field to use for conversation extraction
if 'messages' in sample:
    print("✅ Found 'messages' field - Perfect for conversation data!")
    messages = sample['messages']
    if isinstance(messages, list) and len(messages) > 0:
        print(f"📝 Messages count: {len(messages)}")
        for i, msg in enumerate(messages[:3]):  # Show first 3 messages
            if isinstance(msg, dict):
                role = msg.get('role', 'unknown')
                content = msg.get('content', str(msg))
                content_preview = content[:100] if len(content) > 100 else content
                print(f"  Message {i+1}: {role} - {content_preview}...")

if 'prompt' in sample:
    prompt_preview = sample['prompt'][:200] if len(sample['prompt']) > 200 else sample['prompt']
    print(f"\n✅ Found 'prompt' field: {prompt_preview}...")

print(f"\n🎯 Dataset type identified: {dataset_info['name']}")
print("✅ Ready for Llama conversation processing!")

: 

In [ ]:
# Cell 3 — Llama-3.1 Instruction Dataset Processing (UPDATED FOR ACTUAL STRUCTURE)
import ast
import string
from collections import Counter

# Llama-3.1 instruction format
SYSTEM_PROMPT = """You are an expert at rewriting formal, robotic text to sound natural and human. Make the text conversational, warm, and engaging while preserving the original meaning."""

def format_llama_instruction(formal_text, natural_text):
    """Format training data in Llama-3.1 instruction format"""
    instruction = f"Rewrite this formal text to sound more natural and human:\n\n{formal_text}"
    
    conversation = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

{SYSTEM_PROMPT}<|eot_id|><|start_header_id|>user<|end_header_id|>

{instruction}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{natural_text}<|eot_id|>"""
    
    return conversation

def create_formal_version(text):
    """Convert natural text to formal/robotic version (enhanced)"""
    
    # Expand contractions
    contractions = {
        "don't": "do not", "won't": "will not", "can't": "cannot",
        "shouldn't": "should not", "wouldn't": "would not", 
        "couldn't": "could not", "isn't": "is not", "aren't": "are not",
        "wasn't": "was not", "weren't": "were not", "haven't": "have not",
        "hasn't": "has not", "hadn't": "had not", "I'm": "I am",
        "you're": "you are", "we're": "we are", "they're": "they are",
        "it's": "it is", "that's": "that is", "there's": "there is",
        "let's": "let us", "here's": "here is", "what's": "what is"
    }
    
    formal_text = text
    for contraction, expansion in contractions.items():
        formal_text = re.sub(r'\b' + re.escape(contraction) + r'\b', expansion, formal_text, flags=re.IGNORECASE)
    
    # Replace casual words with formal equivalents
    replacements = {
        r'\bget\b': 'obtain', r'\bgot\b': 'obtained', r'\bgetting\b': 'obtaining',
        r'\bmake\b': 'create', r'\bmade\b': 'created', r'\bmaking\b': 'creating',
        r'\bshow\b': 'demonstrate', r'\bshowed\b': 'demonstrated',
        r'\bthink\b': 'believe', r'\bthought\b': 'believed',
        r'\bwant\b': 'desire', r'\bwanted\b': 'desired',
        r'\bneed\b': 'require', r'\bneeded\b': 'required',
        r'\bstart\b': 'commence', r'\bstarted\b': 'commenced',
        r'\bhelp\b': 'assist', r'\bhelped\b': 'assisted',
        r'\bbig\b': 'significant', r'\bhuge\b': 'substantial',
        r'\btiny\b': 'minimal', r'\bsmall\b': 'limited',
        r'\blots of\b': 'numerous', r'\ba lot of\b': 'numerous',
        r'\bgreat\b': 'excellent', r'\bgood\b': 'satisfactory',
        r'\bbad\b': 'unsatisfactory', r'\bawesome\b': 'exceptional'
    }
    
    for pattern, replacement in replacements.items():
        formal_text = re.sub(pattern, replacement, formal_text, flags=re.IGNORECASE)
    
    # Add more formal sentence structure
    formal_text = re.sub(r'\bSo,\b', 'Therefore,', formal_text)
    formal_text = re.sub(r'\bBut\b', 'However,', formal_text)
    formal_text = re.sub(r'\bAlso,\b', 'Additionally,', formal_text)
    
    return formal_text

def extract_text_from_record(record, dataset_name):
    """Extract usable text from different dataset formats - UPDATED FOR ACTUAL STRUCTURE"""
    texts = []
    
    # Handle the actual dataset structure we found
    if "messages" in record:
        messages = record["messages"]
        if isinstance(messages, list):
            for msg in messages:
                if isinstance(msg, dict):
                    role = msg.get("role", "")
                    content = msg.get("content", "")
                    
                    # Extract natural human responses (assistant or user messages)
                    if role in ["assistant", "user"] and content:
                        content = str(content).strip()
                        if len(content.split()) > 8:  # Reasonable length
                            texts.append(content)
    
    elif "prompt" in record:
        prompt = record.get("prompt", "").strip()
        if prompt and len(prompt.split()) > 8:
            texts.append(prompt)
    
    # Fallback for other dataset types
    elif dataset_name == "microsoft/orca-math-word-problems-200k":
        question = record.get("question", "").strip()
        answer = record.get("answer", "").strip()
        if question and len(question.split()) > 8:
            texts.append(question)
        if answer and len(answer.split()) > 8:
            texts.append(answer)
    
    elif dataset_name == "tatsu-lab/alpaca_cleaned":
        instruction = record.get("instruction", "").strip()
        output = record.get("output", "").strip()
        if instruction and len(instruction.split()) > 5:
            texts.append(instruction)
        if output and len(output.split()) > 8:
            texts.append(output)
    
    elif dataset_name == "humarin/chatgpt-paraphrases":
        # Handle paraphrase format
        original = record.get("text", "").strip()
        paraphrases_raw = record.get("paraphrases", [])
        
        if original:
            texts.append(original)
        
        # Parse paraphrases
        if isinstance(paraphrases_raw, str) and paraphrases_raw.startswith('['):
            try:
                paraphrases = ast.literal_eval(paraphrases_raw)
                for para in paraphrases[:2]:
                    if para and len(str(para).split()) > 5:
                        texts.append(str(para))
            except:
                pass
        elif isinstance(paraphrases_raw, list):
            for para in paraphrases_raw[:2]:
                if para and len(str(para).split()) > 5:
                    texts.append(str(para))
    
    return texts

def is_good_training_pair(formal_text, natural_text):
    """Validate training pair quality for Llama"""
    
    # Length checks
    formal_words = len(formal_text.split())
    natural_words = len(natural_text.split())
    
    if formal_words < 8 or natural_words < 8:
        return False, "too_short"
    if formal_words > 200 or natural_words > 200:
        return False, "too_long"
    
    # Should be meaningfully different
    if formal_text.lower() == natural_text.lower():
        return False, "identical"
    
    # Check overlap (should have reasonable similarity)
    formal_words_set = set(formal_text.lower().split())
    natural_words_set = set(natural_text.lower().split())
    overlap = len(formal_words_set & natural_words_set) / len(formal_words_set | natural_words_set)
    
    if overlap < 0.25:
        return False, "too_different"
    if overlap > 0.95:
        return False, "too_similar"
    
    # Quality checks
    if len(formal_text.strip()) < 20 or len(natural_text.strip()) < 20:
        return False, "too_short_chars"
    
    return True, "good"

# Process dataset for Llama training
conversations = []
skipped_counts = Counter()

print("🔄 Creating Llama-3.1 instruction dataset for humanization...")
print(f"📊 Source: {dataset_info['name']}")
print(f"📦 Total records: {len(ds):,}")

processed_count = 0
for i, record in enumerate(ds):
    if len(conversations) >= MAX_SAMPLES:
        break
    
    if i % 1000 == 0 and i > 0:
        print(f"  Processed {i:,} records, created {len(conversations):,} conversations...")
    
    # Extract texts from record using the actual structure
    texts = extract_text_from_record(record, dataset_info['name'])
    
    if not texts:
        skipped_counts["no_text_extracted"] += 1
        continue
    
    for text in texts:
        if len(conversations) >= MAX_SAMPLES:
            break
        
        # Clean text
        text = text.strip()
        if not text:
            continue
        
        # Create formal version
        formal_version = create_formal_version(text)
        
        # Validate pair
        is_valid, reason = is_good_training_pair(formal_version, text)
        if not is_valid:
            skipped_counts[reason] += 1
            continue
        
        # Format as Llama conversation
        conversation = format_llama_instruction(formal_version, text)
        conversations.append(conversation)
        
        processed_count += 1

print(f"\n✅ LLAMA DATASET CREATION COMPLETE!")
print(f"📊 Total conversations: {len(conversations):,}")
print(f"🎯 Target achieved: {len(conversations) >= MAX_SAMPLES}")

print(f"\n📈 PROCESSING STATS:")
total_processed = sum(skipped_counts.values()) + len(conversations)
print(f"  Records processed: {processed_count:,}")
print(f"  Valid conversations: {len(conversations):,}")
print(f"  Success rate: {len(conversations)/max(total_processed,1)*100:.1f}%")

print(f"\n⚠️  SKIPPED REASONS:")
for reason, count in skipped_counts.most_common():
    print(f"  {reason}: {count:,}")

if len(conversations) == 0:
    print(f"\n❌ No valid conversations created!")
    print(f"💡 Dataset structure analysis:")
    print(f"   Available fields: {ds.column_names}")
    sample = ds[0]
    print(f"   Sample record: {sample}")
    raise ValueError("No valid conversations created! Check dataset structure.")

# Quality analysis
avg_length = sum(len(conv) for conv in conversations) / len(conversations)
print(f"\n📊 QUALITY METRICS:")
print(f"  Average conversation length: {avg_length:.0f} characters")
print(f"  Unique conversations: {len(set(conversations)):,}")

# Show sample conversation
print(f"\n📝 SAMPLE LLAMA CONVERSATION:")
print("=" * 80)
sample_conv = conversations[0]
if len(sample_conv) > 1200:
    print(sample_conv[:600])
    print("\n... [middle content truncated] ...\n")
    print(sample_conv[-600:])
else:
    print(sample_conv)
print("=" * 80)

# Split data
from sklearn.model_selection import train_test_split
train_conversations, val_conversations = train_test_split(
    conversations, test_size=0.1, random_state=SEED
)

print(f"\n🎯 DATASET SPLIT:")
print(f"  Training conversations: {len(train_conversations):,}")
print(f"  Validation conversations: {len(val_conversations):,}")
print(f"\n✅ Llama-3.1 dataset ready for tokenization!")

In [ ]:
# Cell 4 — Load Free Instruction Model and Tokenizer
print(f"🚀 Loading {MODEL_NAME}...")

def try_load_model(model_name):
    """Try to load model with error handling and fallbacks"""
    try:
        print(f"🔄 Attempting to load: {model_name}")
        
        # Load tokenizer
        tokenizer = AutoTokenizer.from_pretrained(
            model_name,
            padding_side="left",  # Better for generation
            truncation_side="right",
            trust_remote_code=True
        )
        
        # Add pad token if missing
        if tokenizer.pad_token is None:
            if tokenizer.eos_token is not None:
                tokenizer.pad_token = tokenizer.eos_token
                tokenizer.pad_token_id = tokenizer.eos_token_id
            else:
                tokenizer.add_special_tokens({'pad_token': '[PAD]'})
        
        print(f"✅ Tokenizer loaded successfully!")
        print(f"📏 Vocab size: {len(tokenizer):,}")
        print(f"🏷️  Pad token: '{tokenizer.pad_token}' (ID: {tokenizer.pad_token_id})")
        
        # Load model with optimized settings
        model_kwargs = {
            "torch_dtype": torch.float16 if DEVICE == "cuda" else torch.float32,
            "device_map": "auto" if DEVICE == "cuda" else None,
            "trust_remote_code": True,
            "low_cpu_mem_usage": True
        }
        
        # Add quantization for large models if needed
        if "7B" in model_name or "large" in model_name:
            if torch.cuda.is_available():
                gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
                if gpu_memory < 16:  # Less than 16GB VRAM
                    print("⚠️  Large model + limited memory - using 8bit quantization")
                    try:
                        from transformers import BitsAndBytesConfig
                        quantization_config = BitsAndBytesConfig(
                            load_in_8bit=True,
                            llm_int8_threshold=6.0,
                        )
                        model_kwargs["quantization_config"] = quantization_config
                        print("✅ 8-bit quantization enabled")
                    except ImportError:
                        print("⚠️  bitsandbytes not available. Install with: pip install bitsandbytes")
        
        base_model = AutoModelForCausalLM.from_pretrained(model_name, **model_kwargs)
        
        # Move to device if not using device_map
        if model_kwargs.get("device_map") is None and DEVICE == "cpu":
            base_model = base_model.to(DEVICE)
        
        return tokenizer, base_model, model_name
        
    except Exception as e:
        print(f"❌ Failed to load {model_name}: {e}")
        return None, None, None

# Try to load the selected model, with fallbacks
tokenizer = None
base_model = None
loaded_model_name = None

# First, try the auto-selected model
tokenizer, base_model, loaded_model_name = try_load_model(MODEL_NAME)

# If that fails, try fallbacks
if tokenizer is None:
    print(f"\n🔄 Trying fallback models...")
    
    for fallback_model in FALLBACK_MODELS:
        print(f"\n📦 Trying fallback: {fallback_model}")
        tokenizer, base_model, loaded_model_name = try_load_model(fallback_model)
        
        if tokenizer is not None:
            print(f"✅ Successfully loaded fallback model: {fallback_model}")
            MODEL_NAME = fallback_model  # Update global variable
            break
    
if tokenizer is None or base_model is None:
    raise ValueError("❌ Could not load any model! Check your internet connection and try again.")

print(f"\n🎉 MODEL SUCCESSFULLY LOADED!")
print(f"📦 Model: {loaded_model_name}")
print(f"🔧 Device: {base_model.device if hasattr(base_model, 'device') else 'Unknown'}")
print(f"📊 Model dtype: {base_model.dtype if hasattr(base_model, 'dtype') else 'Unknown'}")

# Calculate parameters
total_params = sum(p.numel() for p in base_model.parameters())
print(f"📈 Total parameters: {total_params:,} ({total_params/1e6:.0f}M)")

# Memory usage
if torch.cuda.is_available():
    memory_used = torch.cuda.memory_allocated() / 1e9
    memory_total = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"💾 GPU memory used: {memory_used:.1f} GB / {memory_total:.1f} GB")
    print(f"📊 Memory usage: {memory_used/memory_total*100:.1f}%")

print(f"\n🎯 MODEL READY FOR LORA CONFIGURATION!")

# Test tokenization
test_text = "Hello, this is a test."
test_tokens = tokenizer(test_text, return_tensors="pt")
print(f"\n🧪 Tokenization test:")
print(f"   Input: '{test_text}'")
print(f"   Tokens: {test_tokens['input_ids'].shape}")
print(f"   Decoded: '{tokenizer.decode(test_tokens['input_ids'][0])}'")

# Update global variables for consistency
MODEL_NAME = loaded_model_name

In [ ]:
# Cell 5 — Optimized LoRA Configuration for Free Models
print(f"🔧 Configuring LoRA for {MODEL_NAME}...")

# Prepare model for LoRA training
if hasattr(base_model, 'enable_input_require_grads'):
    base_model.enable_input_require_grads()

# Try quantization preparation if model was quantized
try:
    base_model = prepare_model_for_kbit_training(base_model)
    print("✅ Model prepared for quantized training")
except:
    print("ℹ️  Model preparation skipped (not quantized)")

base_model.train()

# ADAPTIVE LORA CONFIG based on model size
total_params = sum(p.numel() for p in base_model.parameters())
model_size = total_params / 1e6  # Size in millions

# Adjust LoRA config based on model size
if model_size > 1000:  # Large models (1B+)
    lora_r = 32
    lora_alpha = 64
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
elif model_size > 500:  # Medium models (500M-1B)
    lora_r = 16
    lora_alpha = 32
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"]
else:  # Small models (<500M)
    lora_r = 8
    lora_alpha = 16
    target_modules = ["q_proj", "v_proj"]

# Detect model architecture and adjust target modules
model_config = base_model.config
architecture = model_config.architectures[0] if hasattr(model_config, 'architectures') else "unknown"

print(f"🏗️  Detected architecture: {architecture}")
print(f"📏 Model size: {model_size:.0f}M parameters")

# Architecture-specific target modules
if "GPT" in architecture or "Gpt" in architecture:
    # GPT-style models (DialoGPT, GPT-2, etc.)
    target_modules = ["c_attn", "c_proj"]  # GPT attention layers
    print("🎯 Using GPT-style attention modules")
elif "Mistral" in architecture:
    # Mistral models
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
    print("🎯 Using Mistral-style modules")
elif "Llama" in architecture or "TinyLlama" in architecture:
    # Llama-style models
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
    print("🎯 Using Llama-style modules")
else:
    # Generic approach - find linear layers
    print("🔍 Detecting linear layers...")
    target_modules = []
    for name, module in base_model.named_modules():
        if isinstance(module, torch.nn.Linear):
            layer_name = name.split('.')[-1]
            if layer_name not in target_modules and len(layer_name) < 20:
                target_modules.append(layer_name)
    
    target_modules = target_modules[:6]  # Limit to 6 most common
    print(f"🎯 Auto-detected modules: {target_modules}")

# Create LoRA configuration
lora_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    target_modules=target_modules,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
    inference_mode=False
)

print(f"\n📋 OPTIMIZED LORA CONFIGURATION:")
print(f"  Model: {MODEL_NAME}")
print(f"  Architecture: {architecture}")
print(f"  Model size: {model_size:.0f}M parameters")
print(f"  LoRA rank (r): {lora_config.r}")
print(f"  LoRA alpha: {lora_config.lora_alpha}")
print(f"  LoRA dropout: {lora_config.lora_dropout}")
print(f"  Target modules: {lora_config.target_modules}")

# Apply LoRA to model
print(f"\n🔄 Applying LoRA adapters...")
try:
    model = get_peft_model(base_model, lora_config)
    print("✅ LoRA adapters applied successfully!")
except Exception as e:
    print(f"❌ Error applying LoRA: {e}")
    print("🔧 Trying with reduced target modules...")
    
    # Fallback with minimal target modules
    lora_config.target_modules = target_modules[:2]  # Use only first 2 modules
    try:
        model = get_peft_model(base_model, lora_config)
        print(f"✅ LoRA applied with reduced modules: {lora_config.target_modules}")
    except Exception as e2:
        print(f"❌ LoRA failed completely: {e2}")
        raise

model.train()

# Enable gradient checkpointing for memory efficiency
try:
    model.gradient_checkpointing_enable()
    print("✅ Gradient checkpointing enabled")
except:
    print("ℹ️  Gradient checkpointing not available")

# Calculate parameter statistics
trainable_params = 0
total_params = 0

for name, param in model.named_parameters():
    total_params += param.numel()
    if param.requires_grad:
        trainable_params += param.numel()

print(f"\n📊 PARAMETER ANALYSIS:")
print(f"  Total parameters: {total_params:,} ({total_params/1e6:.0f}M)")
print(f"  Trainable parameters: {trainable_params:,} ({trainable_params/1e6:.1f}M)")
print(f"  Trainable percentage: {100 * trainable_params / total_params:.3f}%")
print(f"  Memory efficiency: {total_params/trainable_params:.1f}x reduction")

# Validation checks
assert trainable_params > 0, "❌ ERROR: No trainable parameters found!"
assert model.training, "❌ ERROR: Model is not in training mode!"

print(f"\n🎯 LORA SUMMARY:")
model.print_trainable_parameters()

# Memory check
if torch.cuda.is_available():
    current_memory = torch.cuda.memory_allocated() / 1e9
    max_memory = torch.cuda.max_memory_allocated() / 1e9
    total_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    
    print(f"\n💾 MEMORY STATUS:")
    print(f"  Current: {current_memory:.1f} GB / {total_memory:.1f} GB")
    print(f"  Peak: {max_memory:.1f} GB")
    print(f"  Usage: {current_memory/total_memory*100:.1f}%")
    
    # Clear cache
    torch.cuda.empty_cache()

print(f"\n✅ LORA SETUP COMPLETE!")
print(f"🚀 {MODEL_NAME} ready for humanization training!")
print(f"🎯 Trainable parameters: {trainable_params:,} ({trainable_params/1e6:.1f}M)")

# Quick generation test
print(f"\n🧪 Generation test...")
test_input = tokenizer("Hello, how", return_tensors="pt").to(model.device)
try:
    with torch.no_grad():
        test_output = model.generate(
            **test_input, 
            max_length=test_input.input_ids.shape[1] + 5,
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id
        )
    test_result = tokenizer.decode(test_output[0], skip_special_tokens=True)
    print(f"Test result: '{test_result}'")
    print("✅ Model generation verified!")
except Exception as e:
    print(f"⚠️  Generation test failed: {e}")
    print("ℹ️  This may be normal - training should still work")

In [ ]:
# Cell 6 — Llama-3.1 Instruction Dataset Tokenization
from torch.utils.data import Dataset as TorchDataset

class LlamaInstructionDataset(TorchDataset):
    """Custom dataset for Llama instruction fine-tuning"""
    
    def __init__(self, conversations, tokenizer, max_length):
        self.conversations = conversations
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.conversations)
    
    def __getitem__(self, idx):
        conversation = self.conversations[idx]
        
        # Tokenize the full conversation
        encoded = self.tokenizer(
            conversation,
            truncation=True,
            max_length=self.max_length,
            padding="max_length",
            return_tensors="pt"
        )
        
        input_ids = encoded["input_ids"].squeeze()
        attention_mask = encoded["attention_mask"].squeeze()
        
        # For causal LM, labels are the same as input_ids
        # We'll mask the instruction part during loss calculation
        labels = input_ids.clone()
        
        # Find the assistant response start
        # Look for the pattern: assistant<|end_header_id|>\n\n
        assistant_start_pattern = "<|start_header_id|>assistant<|end_header_id|>"
        
        try:
            # Find where assistant response starts
            conversation_text = conversation
            assistant_pos = conversation_text.find(assistant_start_pattern)
            if assistant_pos != -1:
                # Find the actual response start (after the header)
                response_start = conversation_text.find("\n\n", assistant_pos) + 2
                
                # Tokenize just the part before the response
                prefix_text = conversation_text[:response_start]
                prefix_tokens = self.tokenizer(prefix_text, add_special_tokens=False)["input_ids"]
                prefix_length = len(prefix_tokens)
                
                # Mask the instruction part (set to -100 so it's ignored in loss)
                labels[:prefix_length] = -100
        except:
            # If parsing fails, mask first 70% (rough estimate)
            mask_length = int(len(labels) * 0.7)
            labels[:mask_length] = -100
        
        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels
        }

# Create datasets
print("🔄 Creating Llama instruction datasets...")

train_dataset = LlamaInstructionDataset(
    train_conversations,
    tokenizer,
    MAX_LENGTH
)

val_dataset = LlamaInstructionDataset(
    val_conversations,
    tokenizer,
    MAX_LENGTH
)

print(f"✅ DATASETS CREATED:")
print(f"  Training samples: {len(train_dataset):,}")
print(f"  Validation samples: {len(val_dataset):,}")
print(f"  Max sequence length: {MAX_LENGTH}")

# Test tokenization
print(f"\n🔍 TOKENIZATION TEST:")
sample = train_dataset[0]
print(f"  Input shape: {sample['input_ids'].shape}")
print(f"  Attention mask shape: {sample['attention_mask'].shape}")
print(f"  Labels shape: {sample['labels'].shape}")

# Check how much of the sequence is masked
labels = sample['labels']
masked_tokens = (labels == -100).sum().item()
total_tokens = len(labels)
response_tokens = total_tokens - masked_tokens

print(f"  Total tokens: {total_tokens}")
print(f"  Masked tokens (instruction): {masked_tokens}")
print(f"  Response tokens (for training): {response_tokens}")
print(f"  Response percentage: {response_tokens/total_tokens*100:.1f}%")

# Show a sample of the tokenized conversation
print(f"\n📝 SAMPLE TOKENIZED CONVERSATION:")
input_text = tokenizer.decode(sample['input_ids'], skip_special_tokens=True)
print("=" * 80)
print(input_text[:500])
if len(input_text) > 500:
    print("... (truncated)")
print("=" * 80)

# Show where the training starts (non-masked labels)
non_masked_indices = (sample['labels'] != -100).nonzero(as_tuple=True)[0]
if len(non_masked_indices) > 0:
    start_idx = non_masked_indices[0].item()
    response_tokens = sample['input_ids'][start_idx:start_idx+50]  # First 50 response tokens
    response_text = tokenizer.decode(response_tokens, skip_special_tokens=True)
    print(f"\n🎯 TRAINING TARGET (first 50 tokens of response):")
    print(f"'{response_text}'")

print(f"\n✅ Tokenization complete! Ready for training setup.")

# Memory usage check
if torch.cuda.is_available():
    memory_used = torch.cuda.memory_allocated() / 1e9
    print(f"💾 Current GPU memory: {memory_used:.1f} GB")

In [ ]:
# Cell 7 — Llama-3.1 Training Configuration
# Data collator for causal language modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # No masked language modeling for causal LM
    pad_to_multiple_of=8,  # Optimize for tensor cores
    return_tensors="pt"
)

# LLAMA-3.1 OPTIMIZED TRAINING ARGUMENTS
training_args = TrainingArguments(
    output_dir="./llama3_humanization_lora",
    overwrite_output_dir=True,
    
    # Batch and gradient settings
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION,
    gradient_checkpointing=True,  # Save memory
    
    # Training schedule
    num_train_epochs=EPOCHS,
    learning_rate=LR,
    lr_scheduler_type="cosine",
    warmup_steps=100,
    
    # Evaluation and saving
    eval_strategy="steps",
    eval_steps=250,  # More frequent evaluation
    save_strategy="steps", 
    save_steps=250,
    save_total_limit=3,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    
    # Memory and performance optimization
    fp16=True if DEVICE == "cuda" else False,
    dataloader_pin_memory=True if DEVICE == "cuda" else False,
    dataloader_num_workers=0,  # Avoid multiprocessing issues
    
    # Logging and monitoring
    logging_dir="./logs/llama_humanization",
    logging_steps=50,
    logging_strategy="steps",
    report_to=["tensorboard"],
    
    # Optimization
    optim="adamw_torch",
    weight_decay=0.01,
    max_grad_norm=1.0,
    adam_beta1=0.9,
    adam_beta2=0.999,
    adam_epsilon=1e-8,
    
    # Reproducibility
    seed=SEED,
    data_seed=SEED,
    
    # Advanced settings for Llama
    ddp_find_unused_parameters=False,
    remove_unused_columns=False,
    label_names=["labels"],
    prediction_loss_only=True,
    
    # Disable features that can cause issues with instruction tuning
    push_to_hub=False,
    hub_model_id=None,
    
    # Length and padding
    max_steps=-1,  # Use epochs instead
    ignore_data_skip=True
)

print("🎯 LLAMA-3.1 TRAINING CONFIGURATION:")
print("=" * 60)
print(f"📁 Output directory: {training_args.output_dir}")
print(f"📚 Batch size per device: {training_args.per_device_train_batch_size}")
print(f"🔄 Gradient accumulation: {training_args.gradient_accumulation_steps}")
print(f"⚡ Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"🔥 Epochs: {training_args.num_train_epochs}")
print(f"🧠 Learning rate: {training_args.learning_rate}")
print(f"📊 LR scheduler: {training_args.lr_scheduler_type}")
print(f"🔧 Optimizer: {training_args.optim}")
print(f"💾 FP16: {training_args.fp16}")
print(f"✅ Gradient checkpointing: {training_args.gradient_checkpointing}")

# Calculate training metrics
total_samples = len(train_dataset)
effective_batch_size = training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps
steps_per_epoch = total_samples // effective_batch_size
total_steps = steps_per_epoch * training_args.num_train_epochs

print(f"\n📊 TRAINING SCHEDULE:")
print("=" * 40)
print(f"📦 Training samples: {total_samples:,}")
print(f"📏 Steps per epoch: {steps_per_epoch:,}")
print(f"🎯 Total training steps: {total_steps:,}")
print(f"🔥 Warmup steps: {training_args.warmup_steps}")
print(f"📊 Eval every: {training_args.eval_steps} steps")
print(f"💾 Save every: {training_args.save_steps} steps")
print(f"⏱️  Estimated time: {total_steps * 2:.0f}-{total_steps * 5:.0f} minutes")

print(f"\n💡 LLAMA-3.1 OPTIMIZATION FEATURES:")
print("✅ Instruction-following architecture")
print("✅ High-rank LoRA for better adaptation")
print("✅ Proper attention masking")
print("✅ Gradient checkpointing for memory")
print("✅ Cosine learning rate schedule")
print("✅ Frequent evaluation and saving")

# Memory estimation
if torch.cuda.is_available():
    current_memory = torch.cuda.memory_allocated() / 1e9
    print(f"\n💾 MEMORY STATUS:")
    print(f"Current usage: {current_memory:.1f} GB")
    print(f"Estimated training peak: {current_memory * 1.5:.1f} GB")
    
    gpu_total = torch.cuda.get_device_properties(0).total_memory / 1e9
    if current_memory * 1.5 > gpu_total * 0.9:
        print("⚠️  WARNING: May exceed GPU memory during training!")
        print("💡 Consider reducing batch size or using quantization")

print(f"\n✅ Training configuration complete!")
print(f"🚀 Ready to create Llama trainer!")

In [ ]:
# Cell 8 — Create Trainer for Causal Language Model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainable_params_check = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"✅ TRAINER CREATED SUCCESSFULLY!")
print(f"📊 Trainable parameters: {trainable_params_check:,}")
print(f"📦 Training samples: {len(train_dataset):,}")
print(f"🔬 Validation samples: {len(val_dataset):,}")

print(f"📏 Steps per epoch: {len(train_dataset) // (BATCH_SIZE * GRADIENT_ACCUMULATION)}")
print(f"🎯 Total training steps: {(len(train_dataset) // (BATCH_SIZE * GRADIENT_ACCUMULATION)) * EPOCHS}")

In [ ]:
# Cell 9 — Debug Quick-Train (Smoke Test)
print("Starting debug smoke test...")
debug_train = LlamaInstructionDataset(
    train_conversations[:min(180, len(train_conversations))],
    tokenizer,
    MAX_LENGTH
)
debug_val = LlamaInstructionDataset(
    val_conversations[:min(20, len(val_conversations))],
    tokenizer,
    MAX_LENGTH
)

debug_args = TrainingArguments(
    output_dir="./debug_lora_adapter",
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    eval_strategy="no",
    save_strategy="no",
    num_train_epochs=1,
    learning_rate=LR,
    fp16=True if DEVICE == "cuda" else False,
    logging_steps=10,
    seed=SEED,
    dataloader_pin_memory=False,
    remove_unused_columns=False
)

debug_trainer = Trainer(
    model=model,
    args=debug_args,
    train_dataset=debug_train,
    eval_dataset=debug_val,
    tokenizer=tokenizer,
    data_collator=data_collator
)

debug_result = debug_trainer.train()
debug_trainer.save_model("./debug_lora_adapter")

print(f"\nDebug training completed!")
print(f"Final loss: {debug_result.training_loss:.4f}")
if hasattr(debug_result, 'metrics'):
    print(f"Metrics: {debug_result.metrics}")

# Check training losses
state_path = "./debug_lora_adapter/trainer_state.json"
if os.path.exists(state_path):
    with open(state_path, "r") as f:
        state = json.load(f)
    if "log_history" in state:
        losses = [entry.get("loss") for entry in state["log_history"] if "loss" in entry]
        print(f"Last few training losses: {losses[-5:]}")

In [ ]:
# Cell 10 — Full Training
print("Starting full training...")
updated all ceprint(f"Training on {len(train_dataset)} examples")
print(f"Validating on {len(val_dataset)} examples")
print(f"This will take some time...\n")

train_result = trainer.train()

print(f"\nTraining completed!")
print(f"Final training loss: {train_result.training_loss:.4f}")

trainer.save_model("./instruction_lora_humanizer_adapter")
print(f"Adapter saved to: ./instruction_lora_humanizer_adapter")

try:
    model.save_pretrained("./instruction_lora_humanizer_full")
    print(f"Full model saved to: ./instruction_lora_humanizer_full")
except Exception as e:
    print(f"Could not save full model: {e}")

print("\nTraining complete! Adapter ready for inference.")

In [ ]:
# Cell 10.5 — Diagnose Training Failure (Run this if loss is 0.0)
print("=" * 60)
print("TRAINING FAILURE DIAGNOSIS")
print("=" * 60)

print("\n1. CHECK TRAINABLE PARAMETERS:")
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"   Trainable params: {trainable:,}")
if trainable == 0:
    print("   ❌ PROBLEM: No trainable parameters!")
else:
    print("   ✓ Trainable parameters exist")

print("\n2. CHECK DATASET EXTRACTION:")
print(f"   Training samples: {len(train_dataset)}")
print(f"   Validation samples: {len(val_dataset)}")
if len(train_conversations) > 0:
    sample_conv = train_conversations[0]
    # Handle different conversation formats
    if isinstance(sample_conv, dict):
        if 'input' in sample_conv and 'output' in sample_conv:
            print(f"   Sample conversation length: {len(sample_conv['input']) + len(sample_conv['output'])} chars")
            print(f"\n   Input preview: {sample_conv['input'][:150]}...")
            print(f"   Output preview: {sample_conv['output'][:150]}...")
        else:
            print(f"   Sample conversation keys: {list(sample_conv.keys())}")
            print(f"   Sample conversation: {str(sample_conv)[:200]}...")
    else:
        print(f"   Sample conversation type: {type(sample_conv)}")
        print(f"   Sample conversation: {str(sample_conv)[:200]}...")
else:
    print("   No conversations available")
print("\n3. CHECK IF INPUT == OUTPUT (should be different):")
try:
    if len(train_conversations) >= 100:
        if isinstance(train_conversations[0], dict) and 'input' in train_conversations[0]:
            matches = sum(1 for conv in train_conversations[:100] if conv.get('input', '').strip() == conv.get('output', '').strip())
            print(f"   Identical pairs in first 100: {matches}/100")
            if matches > 50:
                print("   ❌ PROBLEM: Too many identical input/output pairs!")
            else:
                print("   ✓ Input and output are different")
        else:
            print("   ℹ️  Cannot check - conversation format not recognized")
    else:
        print("   ℹ️  Less than 100 samples available")
except Exception as e:
    print(f"   ⚠️  Error checking conversations: {e}")
adapter_path = "./instruction_lora_humanizer_adapter"
print("\n4. CHECK ADAPTER FILES:")
import os
adapter_path = "./flan_t5_lora_ultrafeedback_adapter_retrained"
if os.path.exists(adapter_path):
    files = os.listdir(adapter_path)
    adapter_bin = [f for f in files if 'adapter' in f.lower() and '.bin' in f]
    print(f"   Adapter files found: {len(adapter_bin)}")
    if adapter_bin:
        size = os.path.getsize(os.path.join(adapter_path, adapter_bin[0]))
        print(f"   Adapter size: {size:,} bytes")
        if size < 1000:
            print("   ❌ PROBLEM: Adapter file too small!")
        else:
            print("   ✓ Adapter file size looks reasonable")

print("\n" + "=" * 60)
print("\nDIAGNOSIS COMPLETE")
print("If trainable params = 0, the model never trained.")
print("You need to re-run from Cell 5 to re-enable LoRA.")
print("=" * 60)

In [ ]:
# Cell 11 — Inference Load Adapter
print("Loading model for inference...")

dtype = torch.float16 if DEVICE == "cuda" else torch.float32
base_model_inference = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=dtype,
    device_map="auto" if DEVICE == "cuda" else None
)

if DEVICE == "cpu":
    base_model_inference = base_model_inference.to(DEVICE)

model_inference = PeftModel.from_pretrained(
    base_model_inference,
    "./instruction_lora_humanizer_adapter"
)

model_inference.eval()

print(f"Model loaded and ready for inference on {DEVICE}")
print(f"Model is in eval mode: {not model_inference.training}")

In [ ]:
# Cell 12 — Llama-3.1 Humanization Inference Functions
def humanize_with_llama(
    formal_text,
    max_new_tokens=300,
    temperature=0.7,
    top_p=0.9,
    do_sample=True,
    repetition_penalty=1.1
):
    """
    Humanize formal text using fine-tuned Llama-3.1
    
    Args:
        formal_text: Input formal text to humanize
        max_new_tokens: Maximum tokens to generate
        temperature: Sampling temperature (higher = more creative)
        top_p: Nucleus sampling parameter
        do_sample: Whether to use sampling
        repetition_penalty: Penalty for repetitive text
    """
    
    # Create instruction prompt in Llama format
    instruction_prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an expert at rewriting formal, robotic text to sound natural and human. Make the text conversational, warm, and engaging while preserving the original meaning.<|eot_id|><|start_header_id|>user<|end_header_id|>

Rewrite this formal text to sound more natural and human:

{formal_text}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""
    
    # Tokenize input
    inputs = tokenizer(
        instruction_prompt,
        return_tensors="pt",
        truncation=True,
        max_length=MAX_LENGTH - max_new_tokens  # Leave room for response
    ).to(model_inference.device)
    
    # Generate response
    with torch.no_grad():
        outputs = model_inference.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=top_p,
            do_sample=do_sample,
            repetition_penalty=repetition_penalty,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            early_stopping=True
        )
    
    # Decode only the generated part (exclude input prompt)
    input_length = inputs.input_ids.shape[1]
    generated_tokens = outputs[0][input_length:]
    
    # Decode and clean
    response = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    
    # Clean up response
    response = response.strip()
    
    # Remove any remaining special tokens or artifacts
    response = response.replace("<|eot_id|>", "").strip()
    response = response.replace("<|end_of_text|>", "").strip()
    
    return response

def batch_humanize_llama(texts, batch_size=2, **kwargs):
    """Efficiently humanize multiple texts with Llama"""
    results = []
    
    print(f"🔄 Processing {len(texts)} texts in batches of {batch_size}...")
    
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        batch_results = []
        
        for text in batch_texts:
            try:
                result = humanize_with_llama(text, **kwargs)
                batch_results.append(result)
            except Exception as e:
                print(f"⚠️  Error processing text: {e}")
                batch_results.append(f"Error: {text}")
        
        results.extend(batch_results)
        
        if len(results) % 10 == 0:
            print(f"  Completed {len(results)}/{len(texts)}...")
    
    return results

def analyze_humanization_quality(original, humanized):
    """Analyze the quality of humanization"""
    
    # Formality indicators
    formal_indicators = [
        'utilize', 'obtain', 'demonstrate', 'facilitate', 'implement',
        'approximately', 'subsequently', 'furthermore', 'nevertheless',
        'consequently', 'therefore', 'however', 'moreover', 'additionally',
        'commence', 'terminate', 'investigate', 'establish', 'constitute'
    ]
    
    casual_indicators = [
        'use', 'get', 'show', 'help', 'do', 'about', 'then', 'also',
        'but', 'so', 'though', 'and', 'plus', 'too', 'start', 'stop',
        'look into', 'set up', 'make up'
    ]
    
    # Count indicators
    orig_words = original.lower().split()
    hum_words = humanized.lower().split()
    
    orig_formal = sum(1 for word in orig_words if word in formal_indicators)
    orig_casual = sum(1 for word in orig_words if word in casual_indicators)
    
    hum_formal = sum(1 for word in hum_words if word in formal_indicators)
    hum_casual = sum(1 for word in hum_words if word in casual_indicators)
    
    # Calculate metrics
    formality_reduction = orig_formal - hum_formal
    casualness_increase = hum_casual - orig_casual
    length_change = len(hum_words) - len(orig_words)
    
    # Readability estimation (simple)
    avg_word_length_orig = sum(len(word) for word in orig_words) / len(orig_words)
    avg_word_length_hum = sum(len(word) for word in hum_words) / len(hum_words)
    
    return {
        'original_formal_score': orig_formal,
        'humanized_formal_score': hum_formal,
        'original_casual_score': orig_casual,
        'humanized_casual_score': hum_casual,
        'formality_reduction': formality_reduction,
        'casualness_increase': casualness_increase,
        'length_change': length_change,
        'avg_word_length_change': avg_word_length_orig - avg_word_length_hum,
        'humanization_score': (formality_reduction + casualness_increase) / max(len(orig_words), 1) * 100
    }

def compare_responses(formal_text, **generation_kwargs):
    """Compare different generation strategies"""
    
    print(f"🔍 COMPARING GENERATION STRATEGIES")
    print("=" * 60)
    print(f"📝 Original: {formal_text}")
    print()
    
    strategies = [
        {"name": "Conservative", "temperature": 0.3, "do_sample": True},
        {"name": "Balanced", "temperature": 0.7, "do_sample": True},
        {"name": "Creative", "temperature": 1.0, "do_sample": True},
        {"name": "Deterministic", "do_sample": False, "temperature": 1.0}
    ]
    
    results = {}
    
    for strategy in strategies:
        print(f"🎯 {strategy['name']} ({strategy}):")
        try:
            result = humanize_with_llama(formal_text, **{**generation_kwargs, **strategy})
            results[strategy['name']] = result
            print(f"   {result}")
            
            # Quick analysis
            analysis = analyze_humanization_quality(formal_text, result)
            print(f"   📊 Humanization score: {analysis['humanization_score']:.1f}")
        except Exception as e:
            print(f"   ❌ Error: {e}")
            results[strategy['name']] = f"Error: {e}"
        print()
    
    return results

print("🦙 LLAMA-3.1 HUMANIZATION FUNCTIONS READY!")
print("=" * 60)

print(f"\n📋 AVAILABLE FUNCTIONS:")
print("  🎯 humanize_with_llama(text) - Main humanization function")
print("  🎲 humanize_with_llama(text, temperature=1.0) - More creative")
print("  ⚡ batch_humanize_llama(texts) - Process multiple texts")
print("  📊 analyze_humanization_quality(orig, hum) - Quality analysis")
print("  🔍 compare_responses(text) - Test different strategies")

print(f"\n💡 USAGE EXAMPLES:")
print('  result = humanize_with_llama("This implementation demonstrates optimal functionality.")')
print('  results = batch_humanize_llama(["Text 1", "Text 2", "Text 3"])')
print('  analysis = analyze_humanization_quality(original, humanized)')
print('  comparison = compare_responses("Formal text here")')

print(f"\n🎛️  GENERATION PARAMETERS:")
print("  • temperature: 0.1-1.5 (creativity level)")
print("  • top_p: 0.8-0.95 (nucleus sampling)")
print("  • repetition_penalty: 1.0-1.2 (avoid repetition)")
print("  • max_new_tokens: 50-500 (response length)")

print(f"\n✅ Llama-3.1 inference functions ready!")
print("🚀 Perfect for natural, human-like text generation!")

In [ ]:
# Cell 13 — Interactive Testing
test_text = input("Enter text to humanize (or press Enter for sample): ").strip()

if not test_text:
    test_text = "The utilization of artificial intelligence in modern computational systems represents a paradigm shift in technological advancement."
    print(f"Using sample text: {test_text}\n")

print("\n=== CONSERVATIVE (temperature=0.3) ===")
conservative_result = humanize_with_llama(test_text, temperature=0.3)
print(conservative_result)

print("\n=== BALANCED (temperature=0.7) ===")
balanced_result = humanize_with_llama(test_text, temperature=0.7)
print(balanced_result)

print("\n=== CREATIVE (temperature=1.0) ===")
creative_result = humanize_with_llama(test_text, temperature=1.0)
print(creative_result)

print("\n=== ORIGINAL ===")
print(test_text)

In [ ]:
# Cell 14 — Compare Base vs Fine-tuned
comparison_text = "The implementation of machine learning algorithms necessitates comprehensive data preprocessing and feature engineering methodologies."

print("\n=== ORIGINAL TEXT ===")
print(comparison_text)

print("\n=== FINE-TUNED LLAMA OUTPUT ===")
finetuned_result = humanize_with_llama(comparison_text, temperature=0.7)
print(finetuned_result)

print("\n=== ANALYSIS ===")
analysis = analyze_humanization_quality(comparison_text, finetuned_result)
print(f"Humanization score: {analysis['humanization_score']:.1f}")
print(f"Formality reduction: {analysis['formality_reduction']}")
print(f"Casualness increase: {analysis['casualness_increase']}")
print(f"Length change: {analysis['length_change']} words")

print("\nComparison complete!")

In [ ]:
# Cell 15 — Verify Trainable Parameters
total = sum(p.numel() for p in model.parameters())
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Parameter verification:")
print(f"  Total: {total:,}")
print(f"  Trainable: {trainable:,}")
print(f"  Percentage: {100 * trainable / total:.3f}%")

print("\nLoRA modules in model:")
lora_count = 0
for name, module in model.named_modules():
    if "lora" in name.lower():
        print(f"  {name}")
        lora_count += 1
print(f"\nTotal LoRA modules: {lora_count}")

if trainable == 0:
    print("\n⚠️ WARNING: No trainable parameters! Re-enabling LoRA...")
    for name, param in model.named_parameters():
        if any(kw in name.lower() for kw in ["lora", "lora_a", "lora_b"]):
            param.requires_grad = True
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Re-enabled {trainable:,} parameters")

In [ ]:
# Cell 16 — Training Loss History
state_path = "./instruction_lora_humanizer_adapter/trainer_state.json"

if os.path.exists(state_path):
    with open(state_path, "r") as f:
        state = json.load(f)
    
    log_history = state.get("log_history", [])
    losses = [entry.get("loss") for entry in log_history if "loss" in entry]
    
    print(f"Training history loaded: {len(log_history)} entries")
    print(f"Loss entries: {len(losses)}")
    
    if losses:
        print(f"\nLast 10 training losses:")
        for i, loss in enumerate(losses[-10:], 1):
            print(f"  {i}. {loss:.4f}")
        
        print(f"\nFirst loss: {losses[0]:.4f}")
        print(f"Last loss: {losses[-1]:.4f}")
        print(f"Improvement: {losses[0] - losses[-1]:.4f}")
else:
    print(f"Trainer state file not found at {state_path}")

In [ ]:
# Cell 17 — Verify Saved Files
output_dirs = [
    "./instruction_lora_humanizer_adapter",
    "./debug_lora_adapter",
    "./instruction_lora_humanizer_full"
]

for dir_path in output_dirs:
    if os.path.exists(dir_path):
        files = os.listdir(dir_path)
        print(f"\n{dir_path}:")
        print(f"  Files: {len(files)}")
        for f in sorted(files)[:10]:
            size = os.path.getsize(os.path.join(dir_path, f))
            print(f"    {f} ({size:,} bytes)")
        if len(files) > 10:
            print(f"    ... and {len(files) - 10} more files")
    else:
        print(f"\n{dir_path}: NOT FOUND")

In [ ]:
# Cell 18 — Visualize Training Loss
state_path = "./flan_t5_lora_ultrafeedback_retrain/trainer_state.json"

if os.path.exists(state_path):
    with open(state_path, "r") as f:
        state = json.load(f)
    
    log_history = state.get("log_history", [])
    
    steps = []
    losses = []
    eval_losses = []
    eval_steps = []
    
    for entry in log_history:
        if "loss" in entry and "step" in entry:
            steps.append(entry["step"])
            losses.append(entry["loss"])
        if "eval_loss" in entry and "step" in entry:
            eval_steps.append(entry["step"])
            eval_losses.append(entry["eval_loss"])
    
    if losses:
        plt.figure(figsize=(12, 6))
        plt.plot(steps, losses, label="Training Loss", linewidth=2, alpha=0.8)
        if eval_losses:
            plt.plot(eval_steps, eval_losses, label="Validation Loss", linewidth=2, alpha=0.8, marker='o')
        plt.xlabel("Steps", fontsize=12)
        plt.ylabel("Loss", fontsize=12)
        plt.title("Training Progress", fontsize=14, fontweight='bold')
        plt.legend(fontsize=10)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
        
        print(f"\nPlotted {len(losses)} training loss points")
        if eval_losses:
            print(f"Plotted {len(eval_losses)} validation loss points")
    else:
        print("No loss data found to plot")
else:
    print(f"Cannot plot: {state_path} not found")

In [ ]:
# Cell 19 — Fix Trainable Parameters (Emergency Recovery)
print("Emergency LoRA parameter recovery utility")
print("Run this cell if trainable_params becomes 0\n")

before_count = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainable parameters before fix: {before_count:,}")

lora_keywords = ["lora", "lora_a", "lora_b", "lora_embedding", "lora_magnitude"]
fixed_count = 0

for name, param in model.named_parameters():
    if any(keyword in name.lower() for keyword in lora_keywords):
        if not param.requires_grad:
            param.requires_grad = True
            fixed_count += 1
            print(f"  Re-enabled: {name}")

after_count = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\nTrainable parameters after fix: {after_count:,}")
print(f"Parameters re-enabled: {fixed_count}")

if after_count > 0:
    print("\n✓ LoRA parameters successfully recovered!")
    print("  You can now recreate the trainer and continue training.")
else:
    print("\n⚠️ Still no trainable parameters. Check model configuration.")

In [ ]:
# Cell 20 — OOM Recovery Instructions
print("=" * 60)
print("OUT OF MEMORY (OOM) TROUBLESHOOTING")
print("=" * 60)
print("\nIf you encounter CUDA OOM errors, try these fixes:\n")
print("1. REDUCE BATCH SIZE")
print("   Change: BATCH_SIZE = 2  (in Cell 1)")
print("   Then re-run cells 7 and 8 to recreate trainer\n")
print("2. REDUCE MAX LENGTHS")
print("   Change: MAX_SOURCE_LENGTH = 128")
print("   Change: MAX_TARGET_LENGTH = 128")
print("   Then re-run tokenization (Cell 6) and trainer (Cells 7-8)\n")
print("3. REDUCE SAMPLE COUNT")
print("   Change: MAX_SAMPLES = 10000")
print("   Then re-run from Cell 3 onwards\n")
print("4. USE GRADIENT ACCUMULATION")
print("   In Cell 7, add to training_args:")
print("   gradient_accumulation_steps=4")
print("   Then recreate trainer (Cell 8)\n")
print("5. CLEAR CACHE")
print("   Run: torch.cuda.empty_cache()")
print("   Run: import gc; gc.collect()\n")
print("=" * 60)

if torch.cuda.is_available():
    print(f"\nCurrent GPU memory:")
    print(f"  Allocated: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
    print(f"  Reserved: {torch.cuda.memory_reserved() / 1e9:.2f} GB")
    print(f"  Max allocated: {torch.cuda.max_memory_allocated() / 1e9:.2f} GB")

In [ ]:
# Cell 21 — Optional: Hugging Face Login (Commented)
# from huggingface_hub import notebook_login
# notebook_login()
# print("Logged in to Hugging Face!")
# print("This is optional - only needed for private models/datasets")
# print("or to push your fine-tuned model to the Hub")

In [ ]:
# Cell 22 — Llama-3.1 Humanization Complete Setup Summary
print("🦙" * 80)
print("LLAMA-3.1-8B-INSTRUCT HUMANIZATION - SETUP COMPLETE!")
print("🦙" * 80)

print(f"\n🎯 WHY LLAMA-3.1 IS PERFECT FOR HUMANIZATION:")
print("=" * 60)
print("✅ Native instruction understanding - Follows humanization commands naturally")
print("✅ Superior style adaptation - Trained on diverse human writing styles")  
print("✅ Conversational architecture - Designed for natural dialogue")
print("✅ Excellent LoRA adaptation - Fast convergence with fewer examples")
print("✅ Advanced context handling - Understands nuanced rewriting requests")
print("✅ Human preference training - Aligned with human writing preferences")

print(f"\n🤖 MODEL CONFIGURATION:")
print("=" * 40)
print(f"  📦 Base: {MODEL_NAME}")
print(f"  🎯 Task: Formal → Natural human text")
print(f"  🔧 Method: High-rank LoRA (r={LORA_R}, α={LORA_ALPHA})")
print(f"  📏 Context: {MAX_LENGTH:,} tokens")
print(f"  📊 Training samples: {len(train_conversations) if 'train_conversations' in dir() else 'N/A'}")
print(f"  🔬 Validation samples: {len(val_conversations) if 'val_conversations' in dir() else 'N/A'}")

print(f"\n🏃 TRAINING STRATEGY:")
print("=" * 30)
print(f"  🔥 Epochs: {EPOCHS} (Llama learns fast)")
print(f"  📚 Batch: {BATCH_SIZE} × {GRADIENT_ACCUMULATION} = {BATCH_SIZE * GRADIENT_ACCUMULATION} effective")
print(f"  🧠 Learning rate: {LR} (optimized for instruct)")
print(f"  ⚡ Scheduler: Cosine with warmup")
print(f"  💾 Memory: Gradient checkpointing + FP16")
print(f"  🎯 Focus: Instruction following & style transfer")

print(f"\n📊 DATASET ENGINEERING:")
print("=" * 35)
print("  🔄 Smart formal→natural conversion")
print("  📝 Proper Llama instruction format")
print("  🎭 Multiple conversation styles")
print("  ✂️  Intelligent response masking")
print("  🎨 Quality filtering & validation")
print("  📏 Optimal length distribution")

print(f"\n💾 TRAINING OUTPUTS:")
print("=" * 30)
print("  📁 ./llama3_humanization_lora/ (LoRA adapters)")
print("  📊 ./logs/llama_humanization/ (TensorBoard)")
print("  🔍 Model checkpoints every 250 steps")
print("  📈 Evaluation metrics & loss tracking")

print(f"\n🚀 INFERENCE CAPABILITIES:")
print("=" * 35)
print("  🎯 humanize_with_llama() - Natural rewriting")
print("  🎲 Temperature control - Creativity levels")
print("  ⚡ Batch processing - Efficient bulk ops")
print("  📊 Quality analysis - Formality metrics")
print("  🔍 Strategy comparison - A/B testing")

print(f"\n📈 EXPECTED PERFORMANCE:")
print("=" * 35)
print("  ⚡ Fast convergence: ~500-1000 steps")
print("  🎯 High quality: Natural, engaging output")
print("  🔧 Flexible control: Multiple generation modes")
print("  💪 Robust: Handles various text types")
print("  🎨 Creative: Maintains personality & style")

print(f"\n💡 ADVANTAGES OVER T5/FLAN-T5:")
print("=" * 40)
print("  🧠 Better instruction comprehension")
print("  🎭 Superior style & tone adaptation")  
print("  💬 More natural conversational flow")
print("  🚀 Faster training convergence")
print("  🎯 Higher humanization quality")
print("  🔧 More controllable generation")

print(f"\n📋 NEXT STEPS:")
print("=" * 20)
print("  1. 🏃 Run training (Cells 8-10)")
print("  2. 📊 Monitor TensorBoard logs")
print("  3. 🧪 Test humanization (Cell 13)")
print("  4. 🔍 Compare quality (Cell 14)")
print("  5. ⚙️  Fine-tune generation params")
print("  6. 🚀 Deploy for production use")
print("  7. 📈 Scale to larger datasets")

print(f"\n🎊 LLAMA-3.1 HUMANIZATION PIPELINE READY!")
print("=" * 50)
print("ChatGPT was absolutely right - Llama-3.1 is THE model for humanization!")
print("🦙 Start training and experience the difference! 🦙")
print("🦙" * 80)